In [3]:
import requests
import csv
import formatter

def retrieve_games_info(game_name):
    
    game_name = formatter.formatter(game_name)
    # API endpoint to retrieve game details
    endpoint = "https://www.giantbomb.com/api/search"

    # API key, you'll need to sign up for a free API key to use the GiantBomb API
    api_key = ""

    # Query parameters for the API request
    params = {
        "api_key": api_key,
        "format": "json",
        "query": game_name,
        "resources": "game",
        "field_list": "name,expected_release_day,platforms,characters,dlcs,concepts,franchises,genres,regions,releases,description",
        "limit": 1000 # Limit the number of results to 1000
    }

    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept": "*/*",
    "Referer": "https://www.giantbomb.com",
    "Connection": "keep-alive"
    }

    # Send the API request and retrieve the response
    response = requests.get(endpoint, params=params, headers=headers)
    # Check if the response was successful
    if response.status_code == 200:
        # Extract the games information from the response
        games_info = response.json()["results"]
        # Filter the games to include only the ones available on Steam
        #steam_games = [game for game in games_info if "Steam" in [platform["name"] for platform in game["platforms"]]]
        #print(steam_games)
        return games_info

def save_to_csv(games_info, file_name):
    # Write the games information to a CSV file
    with open(file_name, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["Name", "Description", "Expected Release Day", "Original Release Date", "Characters", "DLCs", "Concepts", "Franchises", "Genres", "Regions", "Releases"])
        writer.writeheader()
        for game in games_info:
            writer.writerow({
                "Name": game["name"],
                "Description": game.get("description", ""),
                "Expected Release Day": game.get("expected_release_day", ""),
                "Original Release Date": game.get("original_release_date", ""),
                "Characters": ", ".join([character["name"] for character in game.get("characters", [])]),
                "DLCs": ", ".join([dlc["name"] for dlc in game.get("dlcs", [])]),
                "Concepts": ", ".join([concept["name"] for concept in game.get("concepts", [])]),
                "Franchises": ", ".join([franchise["name"] for franchise in game.get("franchises", [])]),
                "Genres": ", ".join([genre["name"] for genre in game.get("genres", [])]),
                "Regions": ", ".join([region["name"] for region in game.get("regions", [])]),
                "Releases": ", ".join([release["name"] for release in game.get("releases", [])])
            })

# Example usage
games_info = retrieve_games_info("The Elder Scrolls V: Skyrim")
save_to_csv(games_info, "steam_games.csv")